In [5]:
import numpy as np
import cv2
from keras.preprocessing.image import ImageDataGenerator

# Define an image data generator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    fill_mode='nearest',
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255
)

# Define a function to preprocess an individual image
def preprocess_image(image):
    # Check the shape of the input image
    print(image.shape)

    # Convert the image to RGB if it has only one channel
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

    # Resize the image to a smaller size
    image = cv2.resize(image, (256, 256))

    # Normalize the pixel values to a common scale
    image = image / 255.

    return image

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the lapse mechanism
def lapse_mechanism(images, lapse_rate):
    num_images = images.shape[0]
    lapse_mask = np.random.uniform(size=num_images) < lapse_rate
    lapse_values = np.random.uniform(size=num_images)
    return images * np.expand_dims(np.logical_not(lapse_mask), axis=(1, 2, 3)) + np.expand_dims(lapse_values * lapse_mask, axis=(1, 2, 3))

# Load the image data from files
train_datagen = ImageDataGenerator(
    # Data augmentation settings and normalization
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    'brain_tumor_dataset',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)

# Preprocess the image data with a lapse mechanism
x_train, y_train = next(train_generator)
x_train = lapse_mechanism(x_train, lapse_rate=0.1)
x_train = np.array([preprocess_image(image) for image in x_train])

# Define the model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model with a loss function and optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=20)

Found 253 images belonging to 2 classes.
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
Epoch 1/20
32/32 [==============================] - 13s 398ms/step - loss: 0.6931 - accuracy: 0.5625
Epoch 2/20
32/32 [==============================] - 4s 126ms/step - loss: 0.6918 - accuracy: 0.5312
Epoch 3/20
32/32 [==============================] - 4s 136ms/step - loss: 0.6973 - accuracy: 0.5312
Epoch 4/20
32/32 [==============================] - 3s 109ms/step - loss: 0.6914 - accuracy: 0.5312
Epoch 5/20
32/32 [==============================] - 4s 112ms/step - loss: 0.6922 - accuracy: 0.6875
Epoch 6/20

In [6]:
# Save the trained model to a file
model.save('preprocessingKerasModel1.h5')